In [ ]:
#import packages and functions

%load_ext autoreload
%autoreload 2

import pursuit_functions as pursuit
    
import pandas as pd
import numpy as np
import polars as pl
from itertools import product
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm 
from numba import njit

from scipy import stats
from sklearn.decomposition import PCA
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.utils import resample
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay

In [ ]:
#load data sets

RSC_tasks = pd.read_parquet("RSC_sessions.parquet", engine="pyarrow")
PPC_tasks = pd.read_parquet("PPC_sessions.parquet", engine="pyarrow")

# Normalize points and find circle boundaries.

In [ ]:
#get all coordinate values below 99th percentile and normalize points for all regions 

#normalized_sessions = pursuit.tuning.normalize_points(all_pursuit_tasks)
RSC_norm_sessions = pursuit.tuning.normalize_points(RSC_tasks)
PPC_norm_sessions = pursuit.tuning.normalize_points(PPC_tasks)

In [ ]:
#find the mean center and overall radius of the arena for all normalized data points
#you can specify the percentile value to be considered for the overall radius; default is 95th percentile
#calculates the individual center point for each session

#circle_boundaries, radius = pursuit.tuning.fit_circle_bounds(normalized_sessions)
#print(radius)

RSC_circle_boundaries, RSC_radius = pursuit.tuning.fit_circle_bounds(RSC_norm_sessions)
PPC_circle_boundaries, PPC_radius = pursuit.tuning.fit_circle_bounds(PPC_norm_sessions)
print(RSC_radius)
print(PPC_radius)

In [ ]:
#find circumference points for plotting using the center coordinates and overall radius
#all_circ_points = pursuit.tuning.circumference(circle_boundaries)

RSC_circ_points = pursuit.tuning.circumference(RSC_circle_boundaries)
PPC_circ_points = pursuit.tuning.circumference(PPC_circle_boundaries)

# Plot the laser coordinates and boundaries.

In [ ]:
#plot normalized concatenated laser and rat paths with center point and boundary
#the function takes the normalized_sessions, circle_boundaries, and all_circ_points dataframes

pursuit.tuning.plot_arena_bounds(PPC_norm_sessions, PPC_circle_boundaries, PPC_circ_points)

In [ ]:
#plot normalized concatenated laser and rat paths with center point and boundary
#the function takes the normalized_sessions, circle_boundaries, and all_circ_points dataframes

pursuit.tuning.plot_arena_bounds(RSC_norm_sessions, RSC_circle_boundaries, RSC_circ_points)

# Clean data and pull spike data.

In [ ]:
#obtain trial block-specific sessions

RSC_pursuit = RSC_tasks[RSC_tasks["block"] == "pursuit"]
PPC_pursuit = PPC_tasks[PPC_tasks["block"] == "pursuit"]

In [ ]:
#drop NA values for RSC, CA1, and CA3 sessions

RSC_cleaned = pursuit.tuning.drop_NA_vals(RSC_pursuit)
PPC_cleaned = pursuit.tuning.drop_NA_vals(PPC_pursuit)

# Normalizing time for bootstrapping.

In [ ]:
#RSC_clean_time = pursuit.tuning.normalize_time(RSC_cleaned)
#CA1_clean_time = pursuit.tuning.normalize_time(CA1_cleaned)
#CA3_clean_time = pursuit.tuning.normalize_time(CA3_cleaned)

RSC_clean_time = pursuit.tuning.normalize_time(RSC_cleaned)
PPC_clean_time = pursuit.tuning.normalize_time(PPC_cleaned)

# Assigning epochs to dataframes with normalized time.

In [ ]:
# for epoch_half, True:1, False:2 will result in the first half labeled as 1 (<= cutoff) and the second half labeled as 2
# for epoch_odd_even, we take the minutes divided by 2 and find the remainder. If the remainder is 1 (odd), it will be labeled as 1 and if the remainder is 0 (even), it will be labeled as 2

RSC_clean_time_epochs = pursuit.tuning.assign_epochs(RSC_clean_time)
PPC_clean_time_epochs = pursuit.tuning.assign_epochs(PPC_clean_time)

# Create concise dataframes with only sessFile, laser, epoch, spike, and relative time data.

In [ ]:
#make a new df with only sessFile, laser, epoch, spike, and relative time data.

RSC_epoch_laser_spks = pursuit.tuning.epoch_laser_spks(RSC_clean_time_epochs)
PPC_epoch_laser_spks = pursuit.tuning.epoch_laser_spks(PPC_clean_time_epochs)

# Pull epoch halves into two data frames for tuning correlation.

In [ ]:
#make separate dfs for df subsets used for tuning

RSC_epoch_first_half, RSC_epoch_second_half, RSC_epoch_odd_min, RSC_epoch_even_min = pursuit.tuning.pull_epochs(RSC_epoch_laser_spks)
PPC_epoch_first_half, PPC_epoch_second_half, PPC_epoch_odd_min, PPC_epoch_even_min = pursuit.tuning.pull_epochs(PPC_epoch_laser_spks)


# Bootstrapping.

In [ ]:
#bootstrap sessions
#RSC_first_second_tuning = pursuit.tuning.bootstrap_all_sessions(RSC_epoch_first_half, RSC_epoch_second_half, RSC_circle_boundaries)
RSC_odd_even_tuning = pursuit.tuning.bootstrap_all_sessions(RSC_epoch_odd_min, RSC_epoch_even_min, RSC_circle_boundaries)

#PPC_first_second_tuning = pursuit.tuning.bootstrap_all_sessions(PPC_epoch_first_half, PPC_epoch_second_half, PPC_circle_boundaries)
PPC_odd_even_tuning = pursuit.tuning.bootstrap_all_sessions(PPC_epoch_odd_min, PPC_epoch_even_min, PPC_circle_boundaries)


# Find cells with corr values over the 95th and 99th percentiles and compare them to the true tuning values.

In [ ]:
#plot null distribution of spearman r values with true, 95th, 99th percentile lines for all cells
#function takes a tuning epoch df
pursuit.tuning.plot_null_dist(RSC_odd_even_tuning)

In [ ]:
#ID significant cells above 95th percentile corr values
#specify the percentile

#RSC_first_second_cells_95 = pursuit.tuning.get_significant_cells(RSC_first_second_tuning, percentile=95)
RSC_odd_even_cells_95 = pursuit.tuning.get_significant_cells(RSC_odd_even_tuning, percentile=95)
#PPC_first_second_cells_95 = pursuit.tuning.get_significant_cells(PPC_first_second_tuning, percentile=95)
PPC_odd_even_cells_95 = pursuit.tuning.get_significant_cells(PPC_odd_even_tuning, percentile=95)

In [ ]:
#plot null distribution of spearman r values above the 95th percentile for significant cells
#function takes a tuning epoch df, sig cell list 

pursuit.tuning.plot_null_dist(RSC_odd_even_tuning, neurons=RSC_first_second_cells_95)

In [ ]:
#plot null distribution of spearman r values above the 95th percentile for significant cells
pursuit.tuning.plot_null_dist(PPC_odd_even_tuning, neurons=PPC_first_second_cells_95)

# Plot tuning curves for the significant cells.

In [ ]:
#plot tuning heatmap for significant RSC neurons
#function takes the cleaned raw data df, circle boundary df, sig cell list, percentile, smoothing window, exclude_r value (which distances from the center to exclude, and optional plot title)

RSC_df_sorted, RSC_df_binned = pursuit.tuning.sig_cells_heatmap(RSC_cleaned, RSC_circle_boundaries, RSC_odd_even_cells_95, percentile=95, smoothing_window=3, exclude_r=10, plot_title="RSC Max-scored Spike Activity (>95th percentile)")

In [ ]:
#plot tuning heatmap for significant PPC neurons
#function takes the cleaned raw data df, circle boundary df, sig cell list, percentile, smoothing window, exclude_r value (which distances from the center to exclude, and optional plot title)

PPC_df_sorted, PPC_df_binned = pursuit.tuning.sig_cells_heatmap(PPC_cleaned, PPC_circle_boundaries, PPC_odd_even_cells_95, percentile=95, smoothing_window=3, exclude_r=10, plot_title="PPC Max-scored Spike Activity (>95th percentile)")

In [ ]:
#plot the proportion of neurons that peaked in each bin
pursuit.tuning.plot_peaks_by_bin(PPC_df_sorted, RSC_df_sorted, title="PPC and RSC peak bin per neuron")

In [ ]:
#plot individual significant RSC cell tuning curves

pursuit.tuning.plot_sig_tuning_curves(RSC_cleaned, RSC_circle_boundaries, RSC_odd_even_cells_95, percentile=95, exclude_r=10, smoothing_window=3)

In [ ]:
#plot individual significant PPC cell tuning curves

pursuit.tuning.plot_sig_tuning_curves(PPC_cleaned, PPC_circle_boundaries, PPC_odd_even_cells_95, percentile=95, exclude_r=10, smoothing_window=3)

In [ ]:
#plot mean tuning curve for significant RSC cells with SEM 

pursuit.tuning.plot_mean_tuning_curve(RSC_cleaned, RSC_circle_boundaries, RSC_odd_even_cells_95, percentile=95, smoothing_window=3, exclude_r=10, region="RSC")

In [ ]:
#plot mean tuning curve for significant PPC cells with SEM 

pursuit.tuning.plot_mean_tuning_curve(PPC_cleaned, PPC_circle_boundaries, PPC_odd_even_cells_95, percentile=95, smoothing_window=3, exclude_r=10, region="PPC")